In [7]:
import gradio as gr
from langchain_openai import AzureChatOpenAI
import os

# Set up your Azure OpenAI credentials


# Function to read a file and return its content
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Define the initial system prompt
intro_user_prompt = read_file('../Prompts/SystemPrompt-VSC-BusinesCentral-TestRunner.txt')

# Define the initial user messages
user_messages = [
    read_file('../../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al'),
    read_file('../../VSC-BusinessCentral-Test-Runner/package.json'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/apiClient.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/project.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testRunner.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/extension.ts')
]

# Initialize chat history
chat_history = [{"role": "system", "content": "Du bist ein freundliche Assistent und ein Programmierspezialist, der in deutscher Sprache antwortet"},
                {"role": "user", "content": intro_user_prompt}]

# Add user messages to chat history
for msg in user_messages:
    chat_history.append({"role": "user", "content": msg})
    chat_history.append({"role": "assistant", "content": "Datei gespeichert und verstanden. Bitte fortfahren"})

# Function to generate a response from OpenAI
def generate_response(input_text, chat_history):
    chat_history.append({"role": "user", "content": input_text})
    response = AzureChatOpenAI.ChatCompletion.create(
        model="gpt-4o",
        messages=chat_history
    )
    assistant_message = response['choices'][0]['message']['content']
    chat_history.append({"role": "assistant", "content": assistant_message})
    return assistant_message, chat_history

# Function to save chat history
def save_chat_history(chat_history, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(str(chat_history))

# Function to load chat history
def load_chat_history(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return eval(file.read())

# Gradio interface functions
def chat(input_text):
    global chat_history
    assistant_message, chat_history = generate_response(input_text, chat_history)
    return assistant_message

def save_conv():
    global chat_history
    file_path = gr.FileDialog.save_file()
    save_chat_history(chat_history, file_path)

def load_conv():
    global chat_history
    file_path = gr.FileDialog.open_file()
    chat_history = load_chat_history(file_path)
    return chat_history

def delete_all():
    global chat_history
    chat_history = [{"role": "system", "content": system_prompt}]
    return chat_history

def delete_message(index):
    global chat_history
    del chat_history[index]
    return chat_history

# Create Gradio Interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    input_text = gr.Textbox(placeholder="Type your message here...")
    input_button = gr.Button("Submit")
    save_button = gr.Button("Save")
    load_button = gr.Button("Load")
    delete_all_button = gr.Button("Delete All")

    # Display chat history
    for i, message in enumerate(chat_history):
        if message["role"] == "user":
            chatbot.append(message["content"], "user")
        else:
            chatbot.append(message["content"], "assistant")
        delete_button = gr.Button("Delete", elem_id=f"delete_{i}")
        delete_button.click(delete_message, i, chatbot)

    input_button.click(chat, input_text, chatbot)
    save_button.click(save_conv)
    load_button.click(load_conv)
    delete_all_button.click(delete_all)

demo.launch()

AttributeError: 'Chatbot' object has no attribute 'append'